<img src="https://i.imgur.com/OB43U08.jpg">
You are given the below tables, showing student attendance information as well as the grade level (grades 7-8) that each student is in. The columns are labeled in such a way that you should be able to interpret what each field is showing.

__student_attendance_log__

| student_id | attendance_status |
|------------|-------------------|
|57510       |	Present
|57509       |	Present
|57508       |	Tardy
|57507       |	Absent
|57506       |	Present
|57505       |	Present
|57504       |	Present
|57503       |	Tardy
|57501       |	Absent
|57500       |	Present
|57499       |	Tardy
|57498       |	Absent
|57497       | 	Present	

__student_info__

| student_id | grade_level |
|------------|-------------|
|57510       |7
|57509       |7
|57508       |8
|57507       |8
|57506       |7
|57505       |7
|57504       |8
|57503       |8
|57501       |8
|57500       |8
|57499       |7
|57498       |7
|57497       |8

Using the tables above, write a SQL query to return the % of students by grade that are either tardy or absent in your attendance log. As shown in the tables, each student id is unique and exists across both tables.

Your output should return the following columns:

|grade_level |	percent_absent_tardy |	total_num_students
|------------|-----------------------|--------------------|
|7           |0.XX                   |N
|8           |0.YY                   |N

